# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import HTML
from itertools import combinations

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix() + '/src'
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import apply_affine_transform, get_affine_transformation, list_coms, \
    compute_affine_transformation
from library.registration.brain_structure_manager import BrainStructureManager
from library.registration.algorithm import umeyama



from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, read_image, write_image


In [ ]:
animal = 'Allen'
um = 10
affine = False
debug = False
brainManager = BrainStructureManager(animal, um, affine, debug)

In [ ]:
structure = 'SC'
com = np.array([926, 183, 572])
brainManager.update_database_com(structure, com)

In [32]:
def absolute_sum(l):
    la = np.array(l)
    nabs = (np.array(la*la))
    return np.sum(la, axis=0)

def sum_square_com(com):
    ss = np.sqrt(sum([s*s for s in com]))
    return ss

def generate_combinations(lst):
    """
    Generate all combinations of at least 3 elements from the given list.
    
    :param lst: List of elements
    :return: List of tuples containing the combinations
    """
    return list(combinations(lst, 5 ))

    #result = []
    #for r in range(3, len(lst) + 1):
    #    result.extend(combinations(lst, r))
    #return result

def get_umeyama(atlas_src, allen_src):

    A, t = umeyama(atlas_src.T, allen_src.T, with_scaling=True)
    transformation_matrix = np.hstack( [A, t ])
    transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])

    return transformation_matrix

In [43]:
atlas_all = list_coms('Atlas')
allen_all = list_coms('Allen')
common_keys = list(atlas_all.keys() & allen_all.keys())
atlas_common = np.array([atlas_all[s] for s in common_keys])
allen_common = np.array([allen_all[s] for s in common_keys])


In [44]:
combinations_list = generate_combinations(common_keys)
print(len(combinations_list))

435897


## Data Collection

In [65]:
error = {}

for combo in combinations_list[:3000]:
    atlas_src = np.array([atlas_all[s] for s in combo])
    allen_src = np.array([allen_all[s] for s in combo])
    df_list = []
    sss = []
    matrix = compute_affine_transformation(atlas_src, allen_src)
    for structure in common_keys:
        atlas0 = np.array(atlas_all[structure])
        allen0 = np.array(allen_all[structure]) 
        transformed = apply_affine_transform(atlas0, matrix)
        transformed = [round(x,8) for x in transformed]
        difference = [round(a - b, 8) for a, b in zip(transformed, allen0)]
        ss = sum_square_com(difference)
        row = [structure, atlas0, allen0, transformed, difference, ss]
        df_list.append(row)
        sss.append(ss)
    error[combo] = sum(sss)



sorted_combos = {k: v for k, v in sorted(error.items(), key=lambda item: item[1])}
result = list(sorted_combos.items())[:5]
    


In [66]:
result

[(('Sp5I_R', 'SNC_R', 'LRt_R', '4N_L', 'SNC_L'), 1027.2534925291775),
 (('Sp5I_R', 'SNC_R', 'LRt_R', '7N_L', '4N_L'), 1029.4263731973892),
 (('Sp5I_R', 'SNC_R', 'LRt_R', 'SNC_L', '4N_R'), 1043.4174755114302),
 (('Sp5I_R', 'SNC_R', 'LRt_R', '4N_L', 'LRt_L'), 1047.9411140736547),
 (('Sp5I_R', 'SNC_R', 'LRt_R', 'IC', 'SNC_L'), 1048.9575846455537)]

In [84]:
df_list = []
ckeys_single = ('IC', 'AP', 'LRt_R', '7N_L')
atlas_src = np.array([atlas_structures[s] for s in ckeys_single])
allen_src = np.array([allen_structures[s] for s in ckeys_single])
#matrix = get_umeyama(atlas_src, allen_src)
matrix = compute_affine_transformation(atlas_src, allen_src)
error = []
for structure in common_keys:
    atlas0 = np.array(atlas_all[structure])
    allen0 = np.array(allen_all[structure]) 
    transformed = apply_affine_transform(atlas0, matrix)
    transformed = [round(x,2) for x in transformed]
    difference = [round(a - b, 2) for a, b in zip(transformed, allen0)]
    ss = sum_square_com(difference)
    row = [structure, atlas0, allen0, transformed, difference, ss]
    df_list.append(row)
    error.append(ss)
print('SS', sum(error))

SS 2742.442448651634


In [61]:
columns = ['structure', 'atlas0', 'allen0', 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
columns = ['atlas0', 'allen0', 'transformed', 'difference', 'sumsquares']
df.index.name = 'Index'
df.sort_values(by=['structure'], inplace=True)
HTML(df.to_html(index=False))

structure,atlas0,allen0,transformed,difference,sumsquares
3N_L,"[775.63, 391.54, 557.44]","[910.25, 379.44, 552.1]","[914.77, 373.87, 544.25]","[4.52, -5.57, -7.85]",10.633805
3N_R,"[775.63, 391.54, 582.56]","[910.34, 379.4, 586.44]","[912.7, 376.76, 571.04]","[2.36, -2.64, -15.4]",15.801873
4N_L,"[809.24, 396.73, 545.23]","[958.86, 377.78, 543.25]","[946.78, 377.37, 529.64]","[-12.08, -0.41, -13.61]",18.202379
4N_R,"[809.24, 396.73, 594.77]","[958.68, 377.48, 595.29]","[942.71, 383.07, 582.5]","[-15.97, 5.59, -12.79]",21.210212
5N_L,"[866.79, 502.13, 432.89]","[1019.3, 528.65, 409.16]","[1030.35, 499.76, 424.03]","[11.05, -28.89, 14.87]",34.319841
5N_R,"[866.79, 502.13, 707.11]","[1019.34, 528.55, 729.28]","[1007.84, 531.33, 716.59]","[-11.5, 2.78, -12.69]",17.349769
6N_L,"[926.23, 512.4, 533.79]","[1077.12, 521.48, 529.09]","[1077.15, 521.49, 529.09]","[0.03, 0.01, 0.0]",0.031623
6N_R,"[926.23, 512.4, 606.21]","[1077.14, 521.46, 609.41]","[1071.2, 529.83, 606.35]","[-5.94, 8.37, -3.06]",10.710000
7N_L,"[939.45, 615.73, 455.77]","[1085.27, 677.54, 434.0]","[1118.06, 647.64, 463.0]","[32.79, -29.9, 29.0]",53.011264
7N_R,"[939.45, 615.73, 684.23]","[1085.26, 677.53, 704.52]","[1099.3, 673.94, 706.74]","[14.04, -3.59, 2.22]",14.660767


In [79]:
sorted(common_keys)

['3N_L',
 '3N_R',
 '4N_L',
 '4N_R',
 '5N_L',
 '5N_R',
 '6N_L',
 '6N_R',
 '7N_L',
 '7N_R',
 '7n_L',
 '7n_R',
 'AP',
 'Amb_L',
 'Amb_R',
 'DC_L',
 'DC_R',
 'IC',
 'LC_L',
 'LC_R',
 'LRt_L',
 'LRt_R',
 'PBG_L',
 'PBG_R',
 'RtTg',
 'SC',
 'SNC_L',
 'SNC_R',
 'SNR_L',
 'SNR_R',
 'Sp5C_L',
 'Sp5I_L',
 'Sp5I_R',
 'Sp5O_L',
 'Sp5O_R',
 'VLL_L',
 'VLL_R']

In [75]:
counter = 1
for L in range(3, 10):
    for subset in combinations(common_keys, L):
        counter += 1
print(counter)

51228277
